In [ ]:
import os
import sys

from sklearn.model_selection import train_test_split

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))
from SRModels.loading_methods import load_dataset_as_patches
from SRModels.deep_learning_models.SRCNN_model import SRCNNModel
from SRModels.constants import SRCNN_PATCH_SIZE, SRCNN_STRIDE, RANDOM_SEED

In [3]:
HR_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/HR"))
LR_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/LR"))
INTERP_MAP_PATH = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/interpolation_map.pkl"))

In [4]:
# X -> Low-resolution patches (model input) (Low-resolution images with same size as Y but noised)
# Y -> High-resolution patches (target)
X, Y, hr_h, hr_w = load_dataset_as_patches(HR_ROOT, LR_ROOT, mode="srcnn", patch_size=SRCNN_PATCH_SIZE, stride=SRCNN_STRIDE, interpolation_map_path=INTERP_MAP_PATH)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, shuffle=True, random_state=RANDOM_SEED)

In [5]:
print(f"X shape: {X.shape}, Y shape: {Y.shape}")
print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_val shape: {X_val.shape}, Y_val shape: {Y_val.shape}")
print(f"X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")

X shape: (2560, 96, 96, 3), Y shape: (2560, 96, 96, 3)
X_train shape: (1843, 96, 96, 3), Y_train shape: (1843, 96, 96, 3)
X_val shape: (205, 96, 96, 3), Y_val shape: (205, 96, 96, 3)
X_test shape: (512, 96, 96, 3), Y_test shape: (512, 96, 96, 3)


In [15]:
model = SRCNNModel()

model.setup_model(input_shape=X_train.shape[1:], learning_rate=1e-4, loss="mean_squared_error", from_pretrained=False)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 33, 33, 64)        15616     
                                                                 
 conv2d_4 (Conv2D)           (None, 33, 33, 32)        2080      
                                                                 
 conv2d_5 (Conv2D)           (None, 33, 33, 3)         2403      
                                                                 
Total params: 20,099
Trainable params: 20,099
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(X_train, Y_train, X_val, Y_val, batch_size=16, epochs=50, use_augmentation=True, use_mix=True)

Training on GPU: /physical_device:GPU:0
Epoch 1/50
1844/1844 [==============================] - 14s 7ms/step - loss: 0.0145 - psnr: 20.8924 - ssim: 0.4032 - val_loss: 0.0044 - val_psnr: 25.3447 - val_ssim: 0.6127 - lr: 1.0000e-04
Epoch 2/50
1844/1844 [==============================] - 13s 7ms/step - loss: 0.0103 - psnr: 21.7012 - ssim: 0.4186 - val_loss: 0.0045 - val_psnr: 25.3837 - val_ssim: 0.6091 - lr: 1.0000e-04
Epoch 3/50
1844/1844 [==============================] - 12s 7ms/step - loss: 0.0101 - psnr: 21.7360 - ssim: 0.4195 - val_loss: 0.0047 - val_psnr: 24.9143 - val_ssim: 0.6213 - lr: 1.0000e-04
Epoch 4/50
1324/1844 [====================>.........] - ETA: 3s - loss: 0.0101 - psnr: 21.7489 - ssim: 0.4205

KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test, Y_test)

954/954 [==============================] - 4s 4ms/step - loss: 0.0030 - psnr: 27.0695 - ssim: 0.6383
Loss: 0.0030, PSNR: 27.07 dB


[0.002972400514408946, 27.069454193115234, 0.6383132934570312]

In [ ]:
model.save()

Model saved to models/SRCNN\SRCNN_20250624_221949.h5
